In [562]:
# import requests and beautiful soup for parsing
# run 'pip install requests' or 'pip install bs4' if needed
import requests
import bs4
import json
import pandas
import re
import time

# import required modules and set up environment
import os

# replace file path below with your own local convokit
os.chdir('/Users/marianneaubin/Documents/Classes/CS6742/cs6742-fork')

import convokit
from convokit import Corpus, User, Utterance, Parser
from bs4 import NavigableString

In [578]:
def is_last_page(menu):
    counter = menu.text.replace('Results ', '').replace('to ','').replace('of ','')
    comment_counts = counter.split(' ')
    comment_counts = [i.replace(',','') for i in comment_counts]
    comment_counts = [int(i) for i in comment_counts] 
    if comment_counts[1] == comment_counts[2]:
        return True
    return False

In [579]:
def is_last_thread_page(menu):
    counter = menu.text.replace('Threads ', '').replace('to ','').replace('of ','')
    comment_counts = counter.split(' ')
    comment_counts = [i.replace(',','') for i in comment_counts]
    comment_counts = [int(i) for i in comment_counts] 
    if comment_counts[1] == comment_counts[2]:
        return True
    return False

In [569]:
def get_all_urls(soup,threads):
    threads_info = soup.find_all("div", class_="inner")
    for t in threads_info:
        thread = t.find("a", class_="title")
        title = thread.text
        if title != "Non-partisan PSA: machine guns, full-auto, semi-auto, assault weapons...":
            tid = thread["id"]
            threads[tid] = {}
            url = thread["href"].split('.html')
            threads[tid]["url"] = url[0]
            threads[tid]["title"] = title
            meta_info = str(t.find("span", class_="label").text)
            meta_info = (meta_info.replace("By", '').replace(',','')).strip('\xa0').split('\xa0')
            threads[tid]["op"] = meta_info[0]
            threads[tid]["posted_date"] = meta_info[1]            
    return threads

In [570]:
# threads dict will be the central place to keep information about the thread
# it will be indexed by the title id
# the metadata will be the title, the url for access, the date it was posted, 
# the user who posted it, and how many views it has received
# later, we will add the comment and user metadata to each thread as well

threads = {}

page_num = 23
cont = True
while cont:
    url = 'https://www.debatepolitics.com/gun-control/index' + str(page_num) + '.html'
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.text,'lxml')
    if(is_last_thread_page(soup.find("div", class_="threadpagestats"))):
        cont = False
    page_num = page_num + 1
    threads = get_all_urls(soup, threads)
print(threads)

{'thread_title_318364': {'url': 'https://www.debatepolitics.com/gun-control/318364-so-did-shooter-santa-fe-texas-use-shotgun', 'title': 'So, Did the shooter in Santa Fe, Texas use a shotgun?', 'op': 'cuban smokes', 'posted_date': '05-18-18'}, 'thread_title_338364': {'url': 'https://www.debatepolitics.com/gun-control/338364-social-media-password-search-history-could-required-before-buying-firearm-ny', 'title': 'Social media password, search history could be required before buying firearm in NY', 'op': 'Lutherf', 'posted_date': '11-23-18'}, 'thread_title_338801': {'url': 'https://www.debatepolitics.com/gun-control/338801-trump-announce-bump-stock-ban', 'title': 'Trump to announce bump stock ban', 'op': 'calamity', 'posted_date': '11-28-18'}, 'thread_title_338709': {'url': 'https://www.debatepolitics.com/gun-control/338709-florida-group-initiate-assault-weapons-ban-ballot', 'title': 'Florida group to initiate an "assault weapons" ban on the ballot.', 'op': 'Maccabee', 'posted_date': '11-2

In [571]:
num_threads = len(threads.keys())
print("there are", num_threads, "threads")

there are 8 threads


In [580]:
# this dictionary is the uber dictionary
# each thread title is a key 

users = {}
counter = 1

for thread in threads.keys():
    comments = []
    page_num = 1
    cont = True
    print("starting URL ", counter)
    while cont:
        url = threads[thread]['url'] + str(-page_num) + '.html'
        resp = requests.get(url)
        soup = bs4.BeautifulSoup(resp.text,'lxml')
        if(is_last_page(soup.find("div", class_="postpagestats"))):
            cont = False
        page_num = page_num + 1
        users, comments = get_metadata(soup, users, comments)
        threads["comments"] = comments
    print("finished thread ", counter, "out of ", num_threads)
    time.sleep(1)
    counter = counter + 1
print(users)
print(threads)

starting URL  1
finished thread  1 out of  8
starting URL  2
finished thread  2 out of  8
starting URL  3
finished thread  3 out of  8
starting URL  4


ConnectionError: HTTPSConnectionPool(host='www.debatepolitics.com', port=443): Max retries exceeded with url: /gun-control/338709-florida-group-initiate-assault-weapons-ban-ballot-8.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a2a75d748>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [378]:
def get_usernames(soup):
    names = []
    names_raw = soup.find_all("a", class_="username online popupctrl")
    names_raw = names_raw + soup.find_all("a", class_="username offline popupctrl")
    for name_raw in names_raw:
        names.append(name_raw.strong.text)
    return names

print(get_usernames(soup))

['ttwtt78640', 'Lursa', 'TurtleDude', 'Bodhisattva', 'TurtleDude', 'Bodhisattva', 'TurtleDude', 'JMR', 'Lursa', 'JMR']


In [379]:
def extract_username(postinfo):
    name_raw = postinfo.find("a", class_="username offline popupctrl")
    if name_raw is None:
        name_raw = postinfo.find("a", class_="username online popupctrl")
    name = name_raw.strong.text
    return name

In [497]:
def extract_metadata(postinfo):
    jd_raw = postinfo.find("dl", class_="userinfo_extra")
    
    meta = {}
    meta_i = {}
    for i,jd in enumerate(jd_raw):
        if not isinstance(jd, NavigableString):     
            if jd.text == "Join Date":
                meta_i[i+2] = "join_date"
            elif jd.text == "Gender":
                meta_i[i+2] = "gender"
            elif jd.text == "Lean":
                meta_i[i+2] = "lean"
            elif jd.text == "Posts":
                meta_i[i+2] = "posts"
            elif jd.text == "Location":
                meta_i[i+2] = "location"
    for j, jd in enumerate(jd_raw):
        if j in meta_i.keys():
            meta[meta_i[j]] = jd
            
    for elem in meta.keys():
        if elem == "join_date":
            meta[elem] = meta[elem].text
        elif elem == "gender":
            meta[elem] = (meta[elem].find("img")["src"]).replace('https://www.debatepolitics.com/images/misc/','').\
                replace('.gif','')
            if meta[elem] == '':
                meta[elem] = "Unknown"
        elif elem == "lean":
            meta[elem] = meta[elem].text
        elif elem == "posts":
            meta[elem] = int(meta[elem].text.replace(',',''))
        elif elem == "location":
            meta[elem] = meta[elem].text
    
    return meta

In [381]:
def extract_comment(postinfo):
    postedby = postinfo.find("div", class_="bbcode_postedby")
    message = postinfo.find("div", class_="message")
    comment = postinfo.find("blockquote", class_="postcontent restore")
    links = comment.find("a")
    if links != None:
        links.decompose()
    if message != None:
        message.decompose()
    if postedby != None:
        postedby.decompose()
    comment = str(comment.text)
    comment = re.sub('\s\s+', ' ', comment).strip(' ')
    return comment

In [400]:
#import NavigableString

def get_metadata(soup, users_dict, comments_list):
    # meta dict will have a key for each user name
    # the remaining attributes will be in the dict
    # attributes are "join_date", "gender", "lean", "posts", "location", "comment"
    postinfo_list = soup.find_all("div", class_="postdetails")
    
    #for each user
    for postinfo in postinfo_list:
        #extract name of user
        username = extract_username(postinfo)
        #extract metadata
        if username not in users_dict.keys():
            metadata = extract_metadata(postinfo)
            users_dict[username] = {}
            for elem in metadata.keys():
                users_dict[username][elem] = metadata[elem]
        comment = extract_comment(postinfo)
        comments_list.append((username, comment))
    return(users_dict, comments_list)

In [93]:
soup.find_all("div", class_="postdetails")

[<div class="postdetails">
 <div class="userinfo">
 <div class="username_container">
 <div class="popupmenu memberaction">
 <a class="username online popupctrl" href="https://www.debatepolitics.com/members/jacksinpa.html?s=42f4be4a6a237d77488c2302a4249a8f" title="JacksinPA is online now"><strong>JacksinPA</strong></a>
 <ul class="popupbody popuphover memberaction_body">
 <li class="left">
 <a class="siteicon_profile" href="https://www.debatepolitics.com/members/jacksinpa.html?s=42f4be4a6a237d77488c2302a4249a8f">
 				View Profile
 			</a>
 </li>
 <li class="right">
 <a class="siteicon_forum" href="https://www.debatepolitics.com/search.php?do=finduser&amp;userid=32026&amp;contenttype=vBForum_Post&amp;showposts=1&amp;s=42f4be4a6a237d77488c2302a4249a8f" rel="nofollow">
 				View Forum Posts
 			</a>
 </li>
 <li class="left">
 <a class="siteicon_message" href="https://www.debatepolitics.com/private.php?do=newpm&amp;u=32026&amp;s=42f4be4a6a237d77488c2302a4249a8f" rel="nofollow">
 				Privat

In [29]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html dir="ltr" id="vbulletin_html" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <base href="https://www.debatepolitics.com/"/>
  <!--[if IE]></base><![endif]-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="https://www.debatepolitics.com" id="e_vb_meta_bburl" name="vb_meta_bburl"/>
  <meta content="vBulletin 4.2.3" name="generator"/>
  <meta content="IE=9" http-equiv="X-UA-Compatible"/>
  <link href="https://www.debatepolitics.com/favicon.ico" rel="Shortcut Icon" type="image/x-icon"/>
  <script src="https://ajax.googleapis.com/ajax/libs/yui/2.9.0/build/yuiloader-dom-event/yuiloader-dom-event.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   <!--
	if (typeof YAHOO === 'undefined') // Load ALL YUI Local
	{
		document.write('<script type="text/javascript" src="clientscript/yui/yuiloader-

In [250]:
#soup.title.string
#for tag in soup.find_all(True):
    print(tag.name)

html
head
base
meta
meta
meta
meta
link
script
script
script
link
link
link
link
script
script
link
link
link
link
meta
meta
title
link
link
link
script
body
div
div
div
a
img
div
ul
li
a
li
a
li
script
form
fieldset
div
div
input
input
input
input
div
label
input
input
input
input
input
input
script
div
hr
div
ul
li
a
ul
li
a
li
a
li
a
li
a
ul
li
a
li
a
li
a
li
a
ul
li
a
li
a
ul
li
a
li
a
li
a
span
li
a
li
a
div
form
input
input
span
span
input
span
span
input
ul
li
a
div
div
ul
li
a
img
li
a
span
li
a
span
li
a
span
li
a
span
li
span
hr
form
input
input
input
input
input
ol
li
b
b
b
a
b
a
b
div
div
form
input
input
span
a
span
a
span
a
span
a
span
a
span
a
span
span
a
img
span
a
img
ul
li
label
input
input
div
div
h1
span
a
div
div
ul
li
h6
a
ul
li
a
li
li
h6
a
ul
li
label
li
a
li
a
div
ol
li
div
span
span
span
span
a
a
div
div
div
div
a
strong
ul
li
a
li
a
li
a
li
a
img
dd
i
span
span
img
br
img
br
img
br
img
hr
dl
dt
dd
dt
dd
dt
dd
dt
dd
div
div
a
img
a
img
div
div
div
h2
div
div
b